In [163]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

In [164]:
deliveries = pd.read_csv('../data/raw/deliveries.csv')

In [165]:
deliveries.head()

,id,route_id,state,_status,items,total_size,pickupLat,pickupLng,deliveryLat,deliveryLng,pickup_dt,max_pickup_dt,pickup_arrival_dt,pickup_effective_dt,delivery_dt,max_delivery_dt,delivery_arrival_dt,delivery_effective_dt,cost,final_price,distance,shipper_id,distance_charged,distance_route,total_package_price,vehicle_capacity
0,t-amabrz-1563382,r-shiinc-1248022,7,completed,"[{""id"":""AM1001541476YY"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.4531,-44.2639,2022-05-02 13:30:00,2022-05-02 14:30:00,2022-05-02 12:36:34,2022-05-02 13:11:21,2022-05-03 09:21:59,2022-05-03 10:21:59,2022-05-02 20:32:54,2022-05-02 20:35:12,4.46,5.84,70.7,304174.0,0.7,0.7,111.19,2
1,t-amabrz-1563383,r-shiinc-1248059,7,completed,"[{""id"":""AM1001541437YY"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8567,-43.9868,2022-05-02 14:00:00,2022-05-02 15:00:00,2022-05-02 14:25:25,2022-05-02 14:34:37,2022-05-02 15:20:29,2022-05-02 16:20:29,2022-05-02 20:10:09,2022-05-02 20:11:01,2.99,4.95,16.4,131298.0,2.3,2.3,100.42,2
2,t-amabrz-1563384,r-shiinc-1248045,7,completed,"[{""id"":""AM1001541421YY"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8752,-43.9894,2022-05-02 13:30:00,2022-05-02 14:30:00,2022-05-02 13:13:24,2022-05-02 13:44:32,2022-05-02 16:52:01,2022-05-02 17:52:01,2022-05-02 14:54:22,2022-05-02 14:55:32,2.99,4.80,13.3,133352.0,0.0,0.0,70.62,2
3,t-amabrz-1563385,r-shiinc-1248055,7,completed,"[{""id"":""AM1001541484YY"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9114,-43.9134,2022-05-02 13:30:00,2022-05-02 14:30:00,2022-05-02 12:52:37,2022-05-02 13:49:33,2022-05-02 18:38:05,2022-05-02 19:38:05,2022-05-02 17:24:20,2022-05-02 17:27:55,2.99,4.87,15.1,391222.0,0.2,0.2,47.36,2
4,t-amabrz-1563386,r-shiinc-1248066,7,completed,"[{""id"":""AM1001541464YY"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9462,-43.9386,2022-05-02 14:00:00,2022-05-02 15:00:00,2022-05-02 12:34:10,2022-05-02 13:03:52,2022-05-03 04:43:58,2022-05-03 05:43:58,2022-05-02 19:10:47,2022-05-02 19:18:40,2.99,4.80,12.5,140223.0,0.3,0.3,24.90,2


In [166]:
routes_ids_completed=deliveries.route_id.loc[deliveries.state == 7].unique()

In [167]:
routes_totally_complete=[]
for i in routes_ids_completed:
  routes_status=deliveries._status[deliveries.route_id==i].unique()
  if routes_status.size==1 and routes_status[0]=='completed':
    routes_totally_complete.append(i)

In [168]:
deliveries = deliveries.loc[deliveries.route_id.isin( routes_totally_complete)]

In [169]:
deliveries.shape

(51451, 26)

In [170]:
date_fields = [
  "pickup_dt", "max_pickup_dt", "pickup_effective_dt", 
  "delivery_dt", "max_delivery_dt", "delivery_effective_dt"
]
for field in date_fields:
  deliveries = deliveries.loc[deliveries[field] != "0000-00-00 00:00:00"]
  deliveries[field] = pd.to_datetime(deliveries[field])
deliveries.dropna(subset=date_fields, inplace=True)
deliveries.shape

(51451, 26)

In [171]:
deliveries.isna().sum()

id                          0
route_id                    0
state                       0
_status                     0
items                       0
total_size                  0
pickupLat                   0
pickupLng                   0
deliveryLat                 0
deliveryLng                 0
pickup_dt                   0
max_pickup_dt               0
pickup_arrival_dt           0
pickup_effective_dt         0
delivery_dt                 0
max_delivery_dt             0
delivery_arrival_dt        27
delivery_effective_dt       0
cost                        0
final_price                 0
distance                    0
shipper_id                  0
distance_charged            0
distance_route              0
total_package_price      5955
vehicle_capacity            0
dtype: int64

In [172]:
steps = pd.read_csv('../data/raw/steps.csv')

In [173]:
steps.drop_duplicates(inplace=True)

In [174]:
steps.head()

,route_id,delivery_id,action,step_order,distance
0,r-shiinc-1246958,t-petz-3169143,delivery,16,4.3
1,r-shiinc-1246958,t-petz-3169143,pickup,3,0.0
2,r-shiinc-1246958,t-petz-3169633,delivery,14,4.1
3,r-shiinc-1246958,t-petz-3169633,pickup,7,0.0
4,r-shiinc-1246958,t-petz-3169649,delivery,21,2.5


In [175]:
(steps.groupby('route_id')["delivery_id"].count() % 2 != 0).sum()

0

In [176]:
df_grouped_routes_steps = (steps.groupby('route_id')["delivery_id"].count() / 2).reset_index()
df_grouped_routes_steps.columns = ['route_id', 'deliveries_count']
df_grouped_routes_steps['deliveries_count'] = df_grouped_routes_steps['deliveries_count'].astype(int)

In [177]:
df_grouped_routes_deliveries = deliveries.groupby('route_id')["id"].count().reset_index()
df_grouped_routes_deliveries.columns = ['route_id', 'deliveries_count']
df_grouped_routes_deliveries['deliveries_count'] = df_grouped_routes_deliveries['deliveries_count'].astype(int)


In [178]:
df_grouped_routes_steps

,route_id,deliveries_count
0,r-shiinc-1246958,11
1,r-shiinc-1246971,17
2,r-shiinc-1247025,12
3,r-shiinc-1247028,18
4,r-shiinc-1247029,9
...,...,...
1720,r-shiinc-1291913,48
1721,r-shiinc-1293607,63
1722,r-shiinc-1297672,41
1723,r-shiinc-1299464,37


In [179]:
df_grouped_routes_deliveries

,route_id,deliveries_count
0,r-shiinc-1246958,8
1,r-shiinc-1246971,14
2,r-shiinc-1247025,10
3,r-shiinc-1247029,8
4,r-shiinc-1247030,10
...,...,...
1508,r-shiinc-1291913,1
1509,r-shiinc-1293607,1
1510,r-shiinc-1297672,1
1511,r-shiinc-1299464,1


In [180]:
routes_ids = df_grouped_routes_steps.merge(df_grouped_routes_deliveries, on=['route_id', 'deliveries_count'])['route_id'].unique()

In [181]:
deliveries = deliveries.loc[deliveries.route_id.isin(routes_ids)]

In [182]:
deliveries.shape

(26544, 26)

In [183]:
steps = steps.loc[steps.route_id.isin(routes_ids)]

In [184]:
steps.shape

(53088, 5)

In [185]:
# Show all columns in pandas
pd.set_option('display.max_columns', None)

In [186]:
steps = steps.merge(deliveries[['id', 'total_size', 'pickup_effective_dt', 'delivery_effective_dt', 'cost', 'final_price', 'shipper_id']], left_on='delivery_id', right_on='id', how='left')

In [187]:
steps.head()

,route_id,delivery_id,action,step_order,distance,id,total_size,pickup_effective_dt,delivery_effective_dt,cost,final_price,shipper_id
0,r-shiinc-1247126,t-petz-3171108,delivery,10,7.7,t-petz-3171108,5,2022-05-01 16:49:22,2022-05-01 18:02:31,13.9955,15.89,245014.0
1,r-shiinc-1247126,t-petz-3171108,pickup,6,0.0,t-petz-3171108,5,2022-05-01 16:49:22,2022-05-01 18:02:31,13.9955,15.89,245014.0
2,r-shiinc-1247126,t-petz-3171121,delivery,14,2.7,t-petz-3171121,3,2022-05-01 16:49:22,2022-05-01 19:10:33,9.7520,11.30,245014.0
3,r-shiinc-1247126,t-petz-3171121,pickup,2,0.0,t-petz-3171121,3,2022-05-01 16:49:22,2022-05-01 19:10:33,9.7520,11.30,245014.0
4,r-shiinc-1247126,t-petz-3171134,delivery,12,9.0,t-petz-3171134,2,2022-05-01 16:49:22,2022-05-01 18:41:45,15.9390,18.10,245014.0


In [188]:
def get_effective_dt(step):
  if step.action == 'pickup':
    return step['pickup_effective_dt']
  else:
    return step['delivery_effective_dt']

steps['effective_dt'] = steps.apply(get_effective_dt, axis=1)

In [189]:
shippers = pd.read_csv('../data/raw/shippers.csv')

In [190]:
steps = steps.merge(shippers, left_on='shipper_id', right_on='id', how='left')

In [191]:
steps.head()

,route_id,delivery_id,action,step_order,distance,id_x,total_size,pickup_effective_dt,delivery_effective_dt,cost,final_price,shipper_id,effective_dt,id_y,capacity,deliveries_completed
0,r-shiinc-1247126,t-petz-3171108,delivery,10,7.7,t-petz-3171108,5,2022-05-01 16:49:22,2022-05-01 18:02:31,13.9955,15.89,245014.0,2022-05-01 18:02:31,245014,3,343
1,r-shiinc-1247126,t-petz-3171108,pickup,6,0.0,t-petz-3171108,5,2022-05-01 16:49:22,2022-05-01 18:02:31,13.9955,15.89,245014.0,2022-05-01 16:49:22,245014,3,343
2,r-shiinc-1247126,t-petz-3171121,delivery,14,2.7,t-petz-3171121,3,2022-05-01 16:49:22,2022-05-01 19:10:33,9.7520,11.30,245014.0,2022-05-01 19:10:33,245014,3,343
3,r-shiinc-1247126,t-petz-3171121,pickup,2,0.0,t-petz-3171121,3,2022-05-01 16:49:22,2022-05-01 19:10:33,9.7520,11.30,245014.0,2022-05-01 16:49:22,245014,3,343
4,r-shiinc-1247126,t-petz-3171134,delivery,12,9.0,t-petz-3171134,2,2022-05-01 16:49:22,2022-05-01 18:41:45,15.9390,18.10,245014.0,2022-05-01 18:41:45,245014,3,343


In [192]:
route_deliveries_count = (steps.groupby('route_id')['delivery_id'].count() / 2).reset_index()

In [193]:
steps = steps.merge(route_deliveries_count, on='route_id', how='left')

In [194]:
steps

,route_id,delivery_id_x,action,step_order,distance,id_x,total_size,pickup_effective_dt,delivery_effective_dt,cost,final_price,shipper_id,effective_dt,id_y,capacity,deliveries_completed,delivery_id_y
0,r-shiinc-1247126,t-petz-3171108,delivery,10,7.7,t-petz-3171108,5,2022-05-01 16:49:22,2022-05-01 18:02:31,13.9955,15.89,245014.0,2022-05-01 18:02:31,245014,3,343,7.0
1,r-shiinc-1247126,t-petz-3171108,pickup,6,0.0,t-petz-3171108,5,2022-05-01 16:49:22,2022-05-01 18:02:31,13.9955,15.89,245014.0,2022-05-01 16:49:22,245014,3,343,7.0
2,r-shiinc-1247126,t-petz-3171121,delivery,14,2.7,t-petz-3171121,3,2022-05-01 16:49:22,2022-05-01 19:10:33,9.7520,11.30,245014.0,2022-05-01 19:10:33,245014,3,343,7.0
3,r-shiinc-1247126,t-petz-3171121,pickup,2,0.0,t-petz-3171121,3,2022-05-01 16:49:22,2022-05-01 19:10:33,9.7520,11.30,245014.0,2022-05-01 16:49:22,245014,3,343,7.0
4,r-shiinc-1247126,t-petz-3171134,delivery,12,9.0,t-petz-3171134,2,2022-05-01 16:49:22,2022-05-01 18:41:45,15.9390,18.10,245014.0,2022-05-01 18:41:45,245014,3,343,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53083,r-shiinc-1273457,t-amaste-1282632,pickup,13,0.0,t-amaste-1282632,2,2022-05-16 14:29:43,2022-05-17 13:28:32,2.6000,5.76,352819.0,2022-05-16 14:29:43,352819,3,867,60.0
53084,r-shiinc-1273457,t-amaste-1282663,delivery,62,3.0,t-amaste-1282663,2,2022-05-16 14:29:43,2022-05-16 17:04:53,2.6000,4.80,352819.0,2022-05-16 17:04:53,352819,3,867,60.0
53085,r-shiinc-1273457,t-amaste-1282663,pickup,48,0.0,t-amaste-1282663,2,2022-05-16 14:29:43,2022-05-16 17:04:53,2.6000,4.80,352819.0,2022-05-16 14:29:43,352819,3,867,60.0
53086,r-shiinc-1273457,t-amaste-1282704,delivery,98,0.5,t-amaste-1282704,2,2022-05-16 14:29:43,2022-05-16 20:29:20,2.6000,4.87,352819.0,2022-05-16 20:29:20,352819,3,867,60.0


In [195]:
steps.drop(columns=['id_x', 'id_y', 'pickup_effective_dt', 'delivery_effective_dt'], inplace=True)

In [196]:
steps.rename(
  columns={
    'capacity': 'shipper_capacity',
    'deliveries_completed': 'shipper_deliveries_completed',
    'delivery_id_x': 'delivery_id',
    'delivery_id_y': 'route_deliveries_count'
  },
  inplace=True
)

In [197]:
steps.head()

,route_id,delivery_id,action,step_order,distance,total_size,cost,final_price,shipper_id,effective_dt,shipper_capacity,shipper_deliveries_completed,route_deliveries_count
0,r-shiinc-1247126,t-petz-3171108,delivery,10,7.7,5,13.9955,15.89,245014.0,2022-05-01 18:02:31,3,343,7.0
1,r-shiinc-1247126,t-petz-3171108,pickup,6,0.0,5,13.9955,15.89,245014.0,2022-05-01 16:49:22,3,343,7.0
2,r-shiinc-1247126,t-petz-3171121,delivery,14,2.7,3,9.7520,11.30,245014.0,2022-05-01 19:10:33,3,343,7.0
3,r-shiinc-1247126,t-petz-3171121,pickup,2,0.0,3,9.7520,11.30,245014.0,2022-05-01 16:49:22,3,343,7.0
4,r-shiinc-1247126,t-petz-3171134,delivery,12,9.0,2,15.9390,18.10,245014.0,2022-05-01 18:41:45,3,343,7.0


In [198]:
steps.sort_values(by=['route_id', 'step_order'], inplace=True)

In [199]:
steps.reset_index(drop=True, inplace=True)

In [200]:
steps.head()

,route_id,delivery_id,action,step_order,distance,total_size,cost,final_price,shipper_id,effective_dt,shipper_capacity,shipper_deliveries_completed,route_deliveries_count
0,r-shiinc-1247126,t-petz-3171208,pickup,1,0.0,3,10.557,11.98,245014.0,2022-05-01 16:49:22,3,343,7.0
1,r-shiinc-1247126,t-petz-3171121,pickup,2,0.0,3,9.752,11.30,245014.0,2022-05-01 16:49:22,3,343,7.0
2,r-shiinc-1247126,t-petz-3171210,pickup,3,0.0,4,9.752,11.30,245014.0,2022-05-01 16:49:22,3,343,7.0
3,r-shiinc-1247126,t-petz-3171223,pickup,4,0.0,1,13.846,15.72,245014.0,2022-05-01 16:49:22,3,343,7.0
4,r-shiinc-1247126,t-petz-3171209,pickup,5,0.0,5,12.351,14.02,245014.0,2022-05-01 16:49:22,3,343,7.0


In [201]:
def get_next_step_data(step):
  index = step.name
  is_last_step = step.step_order == step.route_deliveries_count * 2
  distance_to_next = 0
  time_to_next = 0
  if not is_last_step:
    distance_to_next = steps.loc[index + 1, 'distance']
    time_to_next = (steps.loc[index + 1, 'effective_dt'] - step.effective_dt).total_seconds()
  return distance_to_next, time_to_next

In [202]:
steps['distance_to_next'],  steps['time_to_next'] = zip(*steps.apply(get_next_step_data, axis=1))

In [203]:
steps.shape

(53088, 15)

In [204]:
steps = steps.loc[steps.time_to_next >= 0]

In [205]:
steps = steps.loc[np.abs(stats.zscore(steps['time_to_next'])) < 3]

In [206]:
steps.shape

(46960, 15)

In [207]:
steps.route_id.nunique()

868

In [210]:
steps.reset_index(drop=True, inplace=True)

In [211]:
steps

,route_id,delivery_id,action,step_order,distance,total_size,cost,final_price,shipper_id,effective_dt,shipper_capacity,shipper_deliveries_completed,route_deliveries_count,distance_to_next,time_to_next
0,r-shiinc-1247126,t-petz-3171208,pickup,1,0.0,3,10.557,11.98,245014.0,2022-05-01 16:49:22,3,343,7.0,0.0,0.0
1,r-shiinc-1247126,t-petz-3171121,pickup,2,0.0,3,9.752,11.30,245014.0,2022-05-01 16:49:22,3,343,7.0,0.0,0.0
2,r-shiinc-1247126,t-petz-3171210,pickup,3,0.0,4,9.752,11.30,245014.0,2022-05-01 16:49:22,3,343,7.0,0.0,0.0
3,r-shiinc-1247126,t-petz-3171223,pickup,4,0.0,1,13.846,15.72,245014.0,2022-05-01 16:49:22,3,343,7.0,0.0,0.0
4,r-shiinc-1247126,t-petz-3171209,pickup,5,0.0,5,12.351,14.02,245014.0,2022-05-01 16:49:22,3,343,7.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46955,r-shiinc-1273881,t-tagros-2545511,pickup,2,0.0,2,9.440,5.95,127284.0,2022-05-17 15:56:51,3,8,3.0,0.0,0.0
46956,r-shiinc-1273881,t-tagros-2548918,pickup,3,0.0,2,9.440,5.95,127284.0,2022-05-17 15:56:51,3,8,3.0,13.8,4252.0
46957,r-shiinc-1273881,t-tagros-2548918,delivery,4,13.8,2,9.440,5.95,127284.0,2022-05-17 17:07:43,3,8,3.0,0.3,61.0
46958,r-shiinc-1273881,t-tagros-2545511,delivery,5,0.3,2,9.440,5.95,127284.0,2022-05-17 17:08:44,3,8,3.0,1.2,480.0
